### 🧩 Préparation et importation des bibliothèques

Dans cette section, nous importons les bibliothèques nécessaires au **prétraitement des images** et à la **préparation des données** pour l’apprentissage automatique :

- **NumPy** → pour la manipulation efficace des tableaux et calculs numériques.  
- **OpenCV (`cv2`)** → pour le traitement d’images (lecture, redimensionnement, filtres, etc.).  
- **`hog` (Histogram of Oriented Gradients)** → pour l’extraction des caractéristiques visuelles (bords, textures, orientations).  
- **`StandardScaler`** → pour normaliser les données et améliorer la convergence des modèles.  
- **`train_test_split`** → pour séparer le jeu de données en ensembles d’entraînement et de test.  
- **`gc`** → pour libérer la mémoire après les traitements intensifs.  
- **`joblib`** → pour sauvegarder et charger efficacement les objets Python volumineux (datasets, modèles, etc.).
- **`Pandas`** → pour la manipulation avancée de données tabulaires, le chargement de datasets, la création de DataFrames, l’exploration, le nettoyage, les jointures et la préparation des données avant l’entraînement des modèles.


In [8]:
import numpy as np
import cv2
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import gc
import joblib
import pandas as pd

### 📂 Lecture des datasets HDF5

Dans cette étape, nous rechargeons les datasets précédemment générés pour chaque **taille d’image** afin de préparer le prétraitement :

- Chaque fichier HDF5 est identifié par : dataset_"taille"_10830.h5

- La clé utilisée pour accéder au DataFrame est : df_"taille"_10830

- Les DataFrames sont stockés dans le dictionnaire `dfs` pour un accès facile selon la taille des images.

> ⚡ Objectif : avoir tous les datasets en mémoire, prêts pour l’extraction des features (HOG) et la normalisation.


In [9]:
# === Lecture du Dataframe depuis HDF5 ===
dfs={}
tailles = [64,128,176,224,299]

for t in tailles:
    filename=f"dataset_{t}_10830.h5"
    key=f"df_{t}_10830"
    dfs[t]=pd.read_hdf(filename,key=key)

### 3️⃣ Prétraitement des images, extraction HOG et création des splits
- Cette cellule effectue plusieurs étapes importantes :
    1. Définition des paramètres HOG et CLAHE pour chaque taille d’image.
    2. Calcul dynamique de la longueur du vecteur HOG selon la taille de l’image, la taille des cellules, des blocs et le nombre d’orientations.
    3. Application de CLAHE sur chaque image pour améliorer le contraste.
    4. Extraction des caractéristiques HOG pour chaque image.
    5. Standardisation des vecteurs de caractéristiques avec StandardScaler.
    6. Séparation train/test avec stratification
    - On utilise `stratify=y_array` pour maintenir la proportion de chaque classe dans l’ensemble d’entraînement et de test.
    - Cela permet d’éviter un déséquilibre dans les splits et d’obtenir des métriques plus fiables lors de l’évaluation des modèles.


    7. Sauvegarde des splits dans des fichiers .pkl pour chaque taille d’image.

💡 Remarque importante : la taille du vecteur HOG varie selon la taille de l’image et le paramétrage des cellules/blocs afin d’avoir un compromis entre détails de l’image et longueur du vecteur.

In [10]:
# Paramètres pour chaque taille d'image
params_dict = {
    64:  {'clahe_clip': 2, 'clahe_grid': (2,2), 'cell': 8,  'bloc': 2, 'orien': 9},
    128: {'clahe_clip': 2, 'clahe_grid': (4,4), 'cell': 12, 'bloc': 2, 'orien': 9},
    176: {'clahe_clip': 2, 'clahe_grid': (5,5), 'cell': 14, 'bloc': 2, 'orien': 9},
    224: {'clahe_clip': 2, 'clahe_grid': (6,6), 'cell': 12, 'bloc': 2, 'orien': 9},
    299: {'clahe_clip': 2, 'clahe_grid': (8,8), 'cell': 10, 'bloc': 1, 'orien': 9}
}


# === Dictionnaires pour stockage - train/test split de chaque taille ===
splits = {}

# Fonction HOG
def extract_hog_features(image,hog_cell,hog_bloc,orien):
    features=hog(image,orientations=orien,
        pixels_per_cell=(hog_cell,hog_cell),
        cells_per_block=(hog_bloc,hog_bloc),
        block_norm='L2-Hys',
        visualize=False)
    return features

def hog_vector_length(img_size, cell, bloc, orien):
    n_cells_x = img_size // cell
    n_cells_y = img_size // cell
    n_blocks_x = n_cells_x - bloc + 1
    n_blocks_y = n_cells_y - bloc + 1
    length = n_blocks_x * n_blocks_y * (bloc ** 2) * orien
    return length

    
# === Boucle principale sur les tailles ===
for size in [64,128,176,224,299]:
    print(f"\n--- Taille {size} ---")

    # Séparation des données
    df=dfs[size]
    X_pixels = df.drop(columns="Cible").to_numpy(dtype=np.uint8)
    y_array = df["Cible"].to_numpy(dtype=np.int32)
    Nb = len(df)

    length = hog_vector_length(size, params_dict[size]['cell'], params_dict[size]['bloc'], params_dict[size]['orien'])
    
    # Préparation arrays
    X_array = np.zeros((Nb, length), dtype=np.float32)

    # Boucle sur chaque image pour HOG 
    for idx, img in enumerate(X_pixels):
        
        # CLAHE augmentation
        clahe = cv2.createCLAHE(clipLimit=params_dict[size]['clahe_clip'],tileGridSize=params_dict[size]['clahe_grid'])
        img_clahe = clahe.apply(img.reshape(size,size))
        
           
    
        # Calcul HOG et retour de l'image “HOG visualisée”
        features=extract_hog_features(img_clahe,params_dict[size]["cell"],params_dict[size]["bloc"], params_dict[size]["orien"])
        X_array[idx, :] = features
    
    
    # StandardScaler avec split pour éviter fuite
    X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.2, random_state=42, stratify=y_array)
    
  
    
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)
    X_test_scaled  = scaler.transform(X_test).astype(np.float32)
    
    # Stockage
    splits[size] = {
        "X_train": X_train_scaled,
        "X_test": X_test_scaled,
        "y_train": y_train,
        "y_test": y_test}

    # Sauvegarde de chaque taille séparément
    joblib.dump(splits[size], f"splits_{size}.pkl", compress=3)
    print(f"✅ Taille {size} terminée et sauvegardée → splits_{size}.pkl")

    
    
    print(f"Taille {size} terminée. Train shape: {X_train_scaled.shape}, Test shape: {X_test_scaled.shape}")

    # Libération mémoire
    del splits[size],X_array, X_train, X_test, X_train_scaled, X_test_scaled
    gc.collect()
    


--- Taille 64 ---
✅ Taille 64 terminée et sauvegardée → splits_64.pkl
Taille 64 terminée. Train shape: (8664, 1764), Test shape: (2166, 1764)

--- Taille 128 ---
✅ Taille 128 terminée et sauvegardée → splits_128.pkl
Taille 128 terminée. Train shape: (8664, 2916), Test shape: (2166, 2916)

--- Taille 176 ---
✅ Taille 176 terminée et sauvegardée → splits_176.pkl
Taille 176 terminée. Train shape: (8664, 4356), Test shape: (2166, 4356)

--- Taille 224 ---
✅ Taille 224 terminée et sauvegardée → splits_224.pkl
Taille 224 terminée. Train shape: (8664, 10404), Test shape: (2166, 10404)

--- Taille 299 ---
✅ Taille 299 terminée et sauvegardée → splits_299.pkl
Taille 299 terminée. Train shape: (8664, 7569), Test shape: (2166, 7569)


### Conclusion sur les vecteurs HOG

On peut remarquer que le vecteur HOG pour les images plus petites que 224 est plus court, car les paramètres sont adaptés à la taille de l’image.  

La taille des cellules et des blocs est choisie pour équilibrer le détail capturé et la longueur du vecteur :  

- Des cellules trop petites donnent trop de valeurs et un vecteur très long.  
- Des cellules trop grandes fournissent moins de gradients et un vecteur trop court.  

Pour les grandes images, on augmente la taille des cellules et ajuste les blocs afin de capturer suffisamment de détails sans rendre le vecteur HOG trop volumineux.

---

## 🚀 Passage à l’entraînement et à l’évaluation des modèles

Les datasets sont désormais **prétraités, normalisés et séparés en train/test** pour chaque taille d’image.  

Dans le notebook suivant, nous allons :  

1. Charger ces splits sauvegardés (`.pkl`) pour chaque taille.  
2. Tester plusieurs modèles classiques et avancés (SVM, RandomForest, GradientBoosting, XGBoost, MLP, etc.).  
3. Comparer les **scores d’entraînement et de test** pour identifier :  
   - Les modèles susceptibles de **surdapprentissage** (train très haut, test faible)  
   - Les modèles en **sous-apprentissage** (scores faibles sur train et test)  
   - Les modèles **équilibrés** offrant de bonnes performances générales  

> 🎯 Objectif : choisir le modèle le plus adapté à nos données avant de passer à l’optimisation fine des hyperparamètres.
